# Brainstorm Auditory Dataset with invertmeeg

This tutorial demonstrates how to use **invertmeeg** solvers on real MEG data from
the [Brainstorm auditory dataset](https://neuroimage.usc.edu/brainstorm/DatasetAuditory).
The dataset contains auditory evoked fields (AEFs) recorded with a 275-channel CTF MEG
system during a standard/deviant oddball paradigm.

We preprocess the data using MNE-Python and then compute inverse solutions with three
solvers from different algorithmic families:

| Solver | Family | Key idea |
|---|---|---|
| **eLORETA** | Minimum Norm | Iteratively reweighted minimum-norm with exact localization |
| **LAURA** | Minimum Norm | Spatially weighted source priors from local autoregression |
| **ESMV** | Beamformer | Eigenspace-projected minimum-variance spatial filter |

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import mne
from mne import combine_evoked
from mne.datasets.brainstorm import bst_auditory
from mne.io import read_raw_ctf

from invert import Solver

## 1. Load and Preprocess the Data

In [ ]:
data_path = bst_auditory.data_path()

subject = "bst_auditory"
subjects_dir = data_path / "subjects"

raw_fname1 = data_path / "MEG" / subject / "S01_AEF_20131218_01.ds"
raw_fname2 = data_path / "MEG" / subject / "S01_AEF_20131218_02.ds"
erm_fname = data_path / "MEG" / subject / "S01_Noise_20131218_01.ds"

# Load and concatenate both runs
raw = read_raw_ctf(raw_fname1)
n_times_run1 = raw.n_times
mne.io.concatenate_raws([raw, read_raw_ctf(raw_fname2)], on_mismatch="ignore")
raw_erm = read_raw_ctf(erm_fname)

raw.set_channel_types({"HEOG": "eog", "VEOG": "eog", "ECG": "ecg"})

### Annotations and artifact projections

We load pre-labeled artifact annotations (bad segments and saccades) and compute
SSP projectors to suppress ocular artifacts.

In [ ]:
# Parse annotations from CSV files
annotations_df = pd.DataFrame()
for idx in [1, 2]:
    csv_fname = data_path / "MEG" / "bst_auditory" / f"events_bad_0{idx}.csv"
    df = pd.read_csv(csv_fname, header=None,
                     names=["onset", "duration", "id", "label"])
    df["onset"] += n_times_run1 * (idx - 1)
    annotations_df = pd.concat([annotations_df, df], axis=0)

saccades_events = df[df["label"] == "saccade"].values[:, :3].astype(int)

annotations = mne.Annotations(
    annotations_df["onset"].values / raw.info["sfreq"],
    annotations_df["duration"].values / raw.info["sfreq"],
    annotations_df["label"].values,
)
raw.set_annotations(annotations)

# Saccade and EOG projectors
saccade_epochs = mne.Epochs(raw, saccades_events, 1, 0.0, 0.5,
                            preload=True, baseline=(None, None),
                            reject_by_annotation=False)
projs_saccade = mne.compute_proj_epochs(saccade_epochs, n_mag=1, n_eeg=0,
                                        desc_prefix="saccade")
proj_fname = data_path / "MEG" / "bst_auditory" / "bst_auditory-eog-proj.fif"
projs_eog = mne.read_proj(proj_fname)[0]
raw.add_proj(projs_saccade)
raw.add_proj(projs_eog)

del saccade_epochs, saccades_events, projs_eog, projs_saccade, annotations_df

### Epoching and averaging

We correct the trigger-to-sound delay using the analog audio channel, reject
bad epochs, and average the standard and deviant conditions separately.

In [ ]:
tmin, tmax = -0.1, 0.5
event_id = dict(standard=1, deviant=2)
reject = dict(mag=4e-12, eog=250e-6)

# Find events and correct trigger delay
events = mne.find_events(raw, stim_channel="UPPT001")
sound_data = raw[raw.ch_names.index("UADC001-4408")][0][0]
onsets = np.where(np.abs(sound_data) > 2.0 * np.std(sound_data))[0]
min_diff = int(0.5 * raw.info["sfreq"])
diffs = np.concatenate([[min_diff + 1], np.diff(onsets)])
onsets = onsets[diffs > min_diff]
assert len(onsets) == len(events)
print(f"Trigger delay removed "
      f"(\u03bc \u00b1 \u03c3): {np.mean(1000.0 * (events[:, 0] - onsets) / raw.info['sfreq']):0.1f}"
      f" \u00b1 {np.std(1000.0 * (events[:, 0] - onsets) / raw.info['sfreq']):0.1f} ms")
events[:, 0] = onsets

# Mark bad channels
raw.info["bads"] = ["MLO52-4408", "MRT51-4408", "MLO42-4408", "MLO43-4408"]

# Create epochs
epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
                    picks=["meg", "eog"], baseline=(None, 0),
                    reject=reject, preload=False, proj=True)
epochs.drop_bad()
epochs.set_annotations(None)

# Balance standard trials and resample
epochs_standard = mne.concatenate_epochs(
    [epochs["standard"][range(40)], epochs["standard"][182:222]]
)
epochs_standard.load_data().resample(600, npad="auto")
epochs_deviant = epochs["deviant"].load_data()
epochs_deviant.resample(600, npad="auto")

# Average and low-pass filter
evoked_std = epochs_standard.average()
evoked_dev = epochs_deviant.average()
for evoked in (evoked_std, evoked_dev):
    evoked.filter(l_freq=None, h_freq=40.0, fir_design="firwin")

del epochs, epochs_standard, epochs_deviant, sound_data

## 2. Evoked Responses

In [ ]:
evoked_std.plot(window_title="Standard", gfp=True, time_unit="s")
evoked_dev.plot(window_title="Deviant", gfp=True, time_unit="s");

In [ ]:
evoked_difference = combine_evoked([evoked_dev, evoked_std], weights=[1, -1])
evoked_difference.plot(window_title="Difference (Deviant - Standard)",
                       gfp=True, time_unit="s");

## 3. Forward Model and Noise Covariance

We create a forward solution using an ico4 source space and estimate a noise covariance
matrix from the empty-room recording.

In [ ]:
# Noise covariance from empty-room recording
cov = mne.compute_raw_covariance(raw_erm, reject=dict(mag=4e-12))
del raw_erm, raw

# Create source space at ico4 resolution
src = mne.setup_source_space(subject, spacing="ico4", subjects_dir=subjects_dir,
                             add_dist="patch")
print(f"Source space: {sum(s['nuse'] for s in src)} sources")

# Create BEM model and solution (single layer for MEG)
conductivity = (0.3,)  # single layer for MEG
model = mne.make_bem_model(subject=subject, subjects_dir=subjects_dir,
                           conductivity=conductivity)
bem = mne.make_bem_solution(model)

# Compute forward solution
fwd = mne.make_forward_solution(evoked_dev.info, trans="fsaverage",
                                src=src, bem=bem, eeg=False, mindist=5.0)
print(f"Forward solution: {fwd['sol']['data'].shape[1]} sources, "
      f"{fwd['sol']['data'].shape[0]} channels")

### Whiten the data using noise covariance

Pre-whitening the data using the noise covariance matrix decorrelates sensor noise
and normalizes channel variances. This improves source localization accuracy,
especially for methods that assume white (identity) noise covariance.

In [ ]:
# Compute whitener from noise covariance
# Pick only magnetometers (CTF system has only axial gradiometers, which MNE treats as "mag")
picks = mne.pick_types(evoked_dev.info, meg="mag", eeg=False, exclude="bads")
ch_names_evoked = [evoked_dev.ch_names[i] for i in picks]

# Find intersection of channels in evoked and covariance
ch_names_cov = cov.ch_names
ch_names_meg = [ch for ch in ch_names_evoked if ch in ch_names_cov]

print(f"Evoked MEG channels: {len(ch_names_evoked)}")
print(f"Covariance channels: {len(ch_names_cov)}")
print(f"Common channels (used for whitening): {len(ch_names_meg)}")

# Pick the common channels from the covariance matrix
cov_picked = mne.cov.pick_channels_cov(cov, ch_names_meg)

# Compute whitener with regularization (rank="info" uses the info to determine rank)
# Create a temporary info with only the selected channels
info_picked = mne.pick_info(evoked_dev.info, mne.pick_channels(evoked_dev.ch_names, ch_names_meg))
whitener, _ = mne.cov.compute_whitener(cov_picked, info_picked, rank="info", )

print(f"Whitener shape: {whitener.shape}")
print(f"Whitener condition number: {np.linalg.cond(whitener):.1f}")

# Apply whitening to evoked data
evoked_std_white = evoked_std.copy().pick(ch_names_meg)
evoked_dev_white = evoked_dev.copy().pick(ch_names_meg)

evoked_std_white.data = whitener @ evoked_std_white.data
evoked_dev_white.data = whitener @ evoked_dev_white.data

# Also whiten the forward model leadfield for consistency
fwd_white = mne.convert_forward_solution(fwd, force_fixed=False)
fwd_white = mne.pick_channels_forward(fwd_white, ch_names_meg)
fwd_white["sol"]["data"] = whitener @ fwd_white["sol"]["data"]

print(f"Whitened evoked data shape: {evoked_dev_white.data.shape}")
print(f"Whitened forward model: {fwd_white['sol']['data'].shape}")

## 4. Source Reconstruction with invertmeeg

Every solver in **invertmeeg** follows the same two-step API:

```python
solver = Solver("name")
solver.make_inverse_operator(fwd, evoked)   # fit
stc = solver.apply_inverse_operator(evoked) # apply
```

We apply each solver to the **deviant** evoked response and plot the result
on the inflated cortical surface at the time of peak activation.

Note: the forward model has ~8k fixed-orientation sources. The solvers below
are all fast enough to handle this comfortably.

In [ ]:
# Shared plotting helper
def plot_source_estimate(stc, title):
    stc.subject = subject
    peak_time = 0.1  # for auditory response
    brain = stc.plot(
        subjects_dir=subjects_dir,
        subject=subject,
        surface="inflated",
        # hemi="lh",
        hemi="both",
        cortex="low_contrast",
        initial_time=peak_time,
        time_unit="s",
        time_viewer=True,
        brain_kwargs=dict(title=title),
        size=(800, 400),
    )
    # brain = stc.plot(
    #     subjects_dir=subjects_dir,
    #     subject=subject,
    #     surface="inflated",
    #     time_viewer=False,
    #     hemi="lh",
    #     initial_time=0.1,
    #     time_unit="s",
    # )
    return brain

In [ ]:
inv = mne.minimum_norm.make_inverse_operator(evoked_std.info, fwd, cov)
snr = 3.0
lambda2 = 1.0 / snr**2


stc_standard = mne.minimum_norm.apply_inverse(evoked_std, inv, lambda2, "dSPM")
brain = stc_standard.plot(
    subjects_dir=subjects_dir,
    subject=subject,
    surface="inflated",
    time_viewer=False,
    hemi="lh",
    initial_time=0.1,
    time_unit="s",
)
del stc_standard, brain

### dSPM

Dynamic Statistical Parametric Mapping (dSPM)

In [ ]:
solver_dspm = Solver("dSPM", depth_weighting=0.5)
solver_dspm.make_inverse_operator(fwd_white, evoked_std_white.copy().crop(0.05, 0.25))
stc_dspm = solver_dspm.apply_inverse_operator(evoked_std_white)
plot_source_estimate(stc_dspm, "dSPM");

### ESMV

Eigenspace-based Minimum Variance beamformer. Projects the data into the
signal subspace before applying a minimum-variance spatial filter, improving
robustness against interference.

In [ ]:
solver_esmv = Solver("esmv")
solver_esmv.make_inverse_operator(fwd_white, evoked_std_white.copy().crop(0.05, 0.25))
stc_esmv = solver_esmv.apply_inverse_operator(evoked_std_white)
plot_source_estimate(stc_esmv, "ESMV");

### APSE
Adaptive Patch Source Estimation (APSE) - A hybrid inverse solution technique
that combines the strengths of beamformers, subspace methods, and sparse
Bayesian approaches for patch-sized source reconstruction.

In [ ]:
solver_esmv = Solver("apse")
solver_esmv.make_inverse_operator(fwd_white, evoked_std_white)#.copy().crop(0.05, 0.2))
stc_esmv = solver_esmv.apply_inverse_operator(evoked_std_white)
plot_source_estimate(stc_esmv, "Adaptive Patch Source Estimation");